In [93]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

#BiocManager::install("treeio")
#BiocManager::install("ggtree")
#BiocManager::install("Biostrings")
#BiocManager::install("XVector")

#install.packages("tidyverse")
#install.packages("ggplot2")
#install.packages("ggstance")
#install.packages("ggmsa")
#install.packages("seqmagick")
#install.packages("shiny")
#install.packages("DT")

also installing the dependencies ‘htmlwidgets’, ‘crosstalk’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [94]:
library(treeio)
library(ggtree)
library(tidyverse)
library(ggplot2)
library(ggstance)
library(Biostrings)
library(ggmsa)
library(seqmagick)
library(XVector)
library(gtable)
library(grid)
library(shiny)
library(DT)


Attaching package: ‘DT’


The following objects are masked from ‘package:shiny’:

    dataTableOutput, renderDataTable




## Likely, the simplest shiny app ever

In [2]:
server <- function(input, output, session) { } #the server

ui <- basicPage("This is a real Shiny app and jupyter is the best thing ever ;D") # the user interface

shinyApp(ui = ui, server = server) # this launches your app


Listening on http://127.0.0.1:7254



## Ok so let's build a web app which displays some trees
## overview: 4 parts - preprocessing, ui, server, app calling

### preporcessing

In [4]:
## define my two classes - defining the A2TEA HYPOTHESES object ;D
# class for the expanded_OG - containing all different types of data we have on it
setClass("expanded_OG", slots=list(genes="spec_tbl_df", 
                                  fasta_files="list", 
                                  msa="AAMultipleAlignment", 
                                  tree="phylo"))

# class for the hypotheses
setClass("hypothesis", slots=list(description="character", 
                                  number="character",
                                  expanded_in ="character", 
                                  compared_to="character", 
                                  expanded_OGs="list"))


#load the A2TEA input
#load("/home/tyll/Desktop/PhD/A2TEA/shiny/example_trees/A2TEA_finished.RData")

### ui

In [113]:
# Define UI for tree app ----

#hypothesesList <- as.list(hypotheses$hypothesis)
#names(hypothesesList) <- hypotheses$name
#hypothesesList


# Allow specific errors to be displayed on screen, instead of displaying a generic error
options(shiny.sanitize.errors = FALSE)

ui <- pageWithSidebar(
  
  # App title ----
  headerPanel("Tree building"),
  
  # Sidebar panel for inputs ----
  sidebarPanel(
  
      fileInput(inputId = "A2TEA",
                label = "Upload A2TEA.RData file:",
                multiple = FALSE,
                accept = ".RData"),
#                accept = ".tsv"), 
      
#      fileInput(inputId = "HYPOTHESES",
#                label = "Upload HYPOTHESES.a2tea.rds file:",
#                multiple = FALSE,
#                accept = ".rds"), 
      
#      fileInput(inputId = "HOG_DE",
#                label = "Upload HOG_DE.a2tea.rds file:",
#                multiple = FALSE,
#                accept = ".rds"), 
      
        # Input: Selector for variable to plot against mpg ----
      selectInput("select",
                    label = h3("Select hypothesis:"),
                    "") 
#                    choices = c("A", "B", "C"))
#                    choices = textOutput("hypothesesList"))
#                    choices = hypothesesList)
  ),
  
  # Main panel for displaying outputs ----
  mainPanel(
      h3(textOutput("text")),
      DT::dataTableOutput("HOG_DE_DT"),
  )
)

In [79]:
?mainPanel

In [71]:
#?selectInput
#create input
#mylist <- as.list(paste0("hypothesis_", hypotheses$hypothesis))
#names(mylist) <- hypotheses$name
#mylist

#accessing the hypotheses
#HYPOTHESES.a2tea$hypothesis_1@expanded_in
#mylist[[1]]
#HYPOTHESES.a2tea[[mylist[[1]]]]@expanded_in

select
#HYPOTHESES.a2tea[[eval(parse(select$input))]]@compared_to

function (.data, ...) 
{
    UseMethod("select")
}
<bytecode: 0x5563ba447f38>
<environment: namespace:dplyr>

### server

In [116]:
#increasing maximum file size for upload
#https://stackoverflow.com/questions/18037737/how-to-change-maximum-upload-size-exceeded-restriction-in-shiny-and-save-user
options(shiny.maxRequestSize=50*1024^2)

myDataFrame <- data.frame(names=c(""))


server <- function(input, output, session) {
    
#    tsv <- reactive({
#     inFile <- input$hypotheses
#     if (is.null(inFile)) {
#         d <- myDataFrame
#     } else {
#         d <- read_tsv(input$hypotheses$datapath)
#        }
#        d
#    })  
 

    tsv <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t = new.env()
    load(input$A2TEA$datapath, envir = t)
    d <- get("hypotheses", envir=t)
     }
     d
   })  
    
    output$hypothesesList <- renderText({
        tsv()$name
    })
    
    observe({
       updateSelectInput(session, "select",
                         label = "Select Hypothesis:",
                         choices = tsv()$name,
                         selected = tsv()$name[1])

    
# also read-in the DE genes table 
    HOG_DE <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t2 = new.env()
    load(input$A2TEA$datapath, envir = t2)
    d <- get("HOG_DE.a2tea", envir = t2)
     }
     d
   }) 

# normal table chokes on large size of df
#    output$HOG_DE <- renderTable({
#        HOG_DE()
#    })
        
    output$HOG_DE_DT <- DT::renderDataTable({
        HOG_DE()
  })
    
#also read-in custom A2TEA object
    HYPOTHESES <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t3 = new.env()
    load(input$A2TEA$datapath, envir = t3)
    d <- get("HYPOTHESES.a2tea", envir = t3)
     }
     d
   })  
    
        
    hypothesisNUM <- reactive({
       HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@number
    })
   
     output$text <- renderText({
        hypothesisNUM()
    })    
        
    
    # read-in the .RData object
#    data <- reactive({
#       readRDS(input$HYPOTHESES$datapath)
#    })
    
#    hypothesisNUM <- reactive({
##        data()[[as.numeric(input$select)]]@number
#        data()[[hypotheses$hypothesis[hypotheses$name==input$select]]]@number
#    })
   
#      output$text <- renderText({
#        hypothesisNUM()
#    })
    
#    output$recFacetPlot <- renderPlot({
        
#  })

    })
    
}

In [232]:
#saveRDS(HYPOTHESES.a2tea, "example_trees/HYPOTHESES.a2tea.rds")
#HYPOTHESES.a2tea[[testSelect]]@compared_to
read_tsv("example_trees/hypotheses.tsv")$name


── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────
cols(
  hypothesis = col_double(),
  name = col_character(),
  expanded_in = col_character(),
  compared_to = col_character()
)




[1] "Expanded in Arabidopsis compared to Monocots"
[2] "Expanded in barley compared to maize"

### app calling

In [117]:
shinyApp(ui, server)


Listening on http://127.0.0.1:4827

Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in [[: attempt to select less than one element in get1index”


### playground & working examples

In [282]:
hypotheses$hypothesis[hypotheses$name=="Expanded in barley compared to maize"]

[1] 2

In [247]:
saveRDS( data.frame(names=c("Jill","Jane","Megan")),"example_trees/myDataFrame.rds")

#### completely working mini application showcasing runtime loading of data and reactive dropdown

In [249]:
myDataFrame <- data.frame(names=c("Tom","Dick","Harry"))

ui <- shinyUI(
    fluidPage(
        fileInput("file1", "Choose file to upload", accept = ".rds"),
        selectInput("myNames","Names", ""),
        tableOutput("contents")
    )
)

server <- function(input, output, session) {

    myData <- reactive({
        inFile <- input$file1
        if (is.null(inFile)) {
            d <- myDataFrame
        } else {
            d <- readRDS(inFile$datapath)
        }
        d
    })

    output$contents <- renderTable({
        myData()
    })

    observe({
         updateSelectInput(session, "myNames",
                           label = "myNames",
                           choices = myData()$names,
                           selected = myData()$names[1])
    })

}

shinyApp(ui, server)



Listening on http://127.0.0.1:5402



#### shiny example of using RData input

In [47]:
#The example allows you to upload .RData files.
#The approach with load and get allows you to assign the loaded data to a variable name of your choice.
#For the matter of the example being "standalone" I inserted the top section that stores 
#two vectors to your disk in order to load and plot them later.


# Define two datasets and store them to disk
x <- rnorm(100)
save(x, file = "x.RData")
#rm(x)
y <- rnorm(100, mean = 2)
save(y, file = "y.RData")
#rm(y)
# both
save(x, y, file = "z.RData")

# This function, borrowed from http://www.r-bloggers.com/safe-loading-of-rdata-files/, load the Rdata into a new environment to avoid side effects
LoadToEnvironment <- function(RData, env=new.env()) {
  load(RData, env)
  return(env)
}


# Define UI
ui <- shinyUI(fluidPage(
  titlePanel(".RData File Upload Test"),
  mainPanel(
    fileInput("file", label = ""),
    actionButton(inputId="plot","Plot"),
    plotOutput("hist"))
  )
)

# Define server logic
server <- shinyServer(function(input, output) {

  observeEvent(input$plot,{
    if ( is.null(input$file)) return(NULL)
    inFile <- input$file
    file <- inFile$datapath
    # load the file into new environment and get it from there
#    name <- load(file, envir = e)
#    data <- e[[name]]
    # this works but not really what I need
#    data <- e[[name[1]]]
    t = new.env()
    load(file, envir = t)
    data <- get("y", envir=t)

      
      
#    observeEvent(input$plot,{
#        if ( is.null(input$file)) return(NULL)
#        inFile <- input$file
#        file <- inFile$datapath
#    # Use a reactiveFileReader to read the file on change, and load the content into a new environment
#    env <- reactiveFileReader(1000, session, "z.RData", LoadToEnvironment)
#    # Access the first item in the new environment, assuming that the Rdata contains only 2 items which is a data frame
#    env()[[names(env())[2]]]
    
    
    # Plot the data
    output$hist <- renderPlot({
      hist(data)
    })
  })
})

# Run the application 
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:4827



In [42]:
e = new.env()
name <- load("z.RData", envir = e)
#mget(c("y","x"), envir=e)
#get returns exactly the object
test <- get("y", envir=e)
str(test)
str(y)

 num [1:100] 3.011 0.694 1.618 1.81 2.27 ...
 num [1:100] 3.011 0.694 1.618 1.81 2.27 ...


In [112]:
#load("example_trees//A2TEA_finished.RData")
#hypotheses[]
#HOG_DE.a2tea
#HYPOTHESES.a2tea